In [1]:
import pandas as pd
import numpy as np
import json

# Health index

Two CSVs are required. They can be downloaded here:
1) https://drive.google.com/file/d/13DD3nb4VR2guEl6M87qjVMcdn1QjWpzC/view?usp=sharing
2) https://drive.google.com/file/d/1Zp2LPEqv6NTdItB1V6osW1oP5SsvbTIc/view?usp=sharing 

Once CSVs are downloaded change the directory paths below

In [2]:
# Read main csv data and drop empty column
df = pd.read_csv('/Users/theojolliffe/Downloads/All data-Table.csv')
df = df.drop(['Unnamed: 7'], axis=1)

# Create empty columns we will populate later
df["Rank"] = np.nan
df["Change1year"] = np.nan
df["Change3year"] = np.nan
df["highestRank"] = np.nan
df["highestRankType"] = np.nan
df["lowestRank"] = np.nan
df["lowestRankType"] = np.nan
df["Change1year Rank"] = np.nan
df["Change3year Rank"] = np.nan
df['value'] = df['Index value']

# Correct spelling mistakes in data
df['Indicator/grouping name'] = df['Indicator/grouping name'].replace({'Local enviroment': 'Local environment'})
df['Indicator/grouping name'] = df['Indicator/grouping name'].replace({'Public greenspace': 'Public green space'})

df["Measure"]=df["Indicator/grouping name"]

# Read mapping csv, used later to create hierarchal structure of data (Domains>Subdomains>Indicators)
indicator_mapping = pd.read_csv('/Users/theojolliffe/Downloads/indicator_mapping.csv')

## Creating the hierarchy
We need to add columns that hold information about which domain and subdomain each data point belongs to

In [3]:
# Create dfs filtered by index level 
subdomains = df[df['Index level'] == 'Subdomain']
domains = df[df['Index level'] == 'Domain']

# Create dictionaries that map indicators to subdomains and subdomains to domains
indicator_domain_mapping = indicator_mapping[['Domain', 'Indicator']].set_index('Indicator').to_dict()['Domain']
indicator_subdomain_mapping = indicator_mapping[['Subdomain', 'Indicator']].set_index('Indicator').to_dict()['Subdomain']
subdomain_domain_mapping = indicator_mapping[['Subdomain', 'Domain']].set_index('Subdomain').to_dict()['Domain']
subdomain_subdomain_mapping = {key: key for key in subdomains['Indicator/grouping name'].unique()}
domain_domain_mapping = {key: key for key in domains['Indicator/grouping name'].unique()}

### Create domain column

In [4]:
# Mapping points function to the correct domain mapping depending on index level of row
domain_replacement_mapping = {'Domain': domain_domain_mapping,
                             'Subdomain': subdomain_domain_mapping,
                             'Indicator': indicator_domain_mapping}

# This function finds the correct domain for any indicator, subdomain or domain
def get_domain(row):
    level = row['Index level']
    # Overall health data is stored alongside the domains
    if level == 'Overall':
        return 'Overall'
    replacement_mapping = domain_replacement_mapping[level]
    return replacement_mapping[row['Indicator/grouping name']]

# Create new column holding domain information
df['Domain'] = df.apply(get_domain, axis=1)

### Create subdomain and indicator columns

In [5]:
# Points to correct subdomain mapping
subdomain_replacement_mapping = {'Subdomain': subdomain_subdomain_mapping,
                             'Indicator': indicator_subdomain_mapping}

# Finds the correct subdomain for any indicator, subdomain or domain (domain = NaN)
def get_subdomain(row):
    level = row['Index level']
    if level in ['Overall', 'Domain']:
        return np.nan
    replacement_mapping = subdomain_replacement_mapping[level]
    return replacement_mapping[row['Indicator/grouping name']]

# Create subdomain column
df['Subdomain'] = df.apply(get_subdomain, axis=1)

# Create indicator column with NaNs for overall, domain, and subdomain rows
df['Indicator'] = df['Indicator/grouping name'].where(df['Index level'] == 'Indicator', np.nan)

## Calculating change over time

In [6]:
# Create a table that holds the change in value from previous year
def get_diffs(group):
    # By default diff() calculates the different between a value in the previous row
    output = group.sort_values('Year')['Index value'].diff()
    output.index = group.sort_values('Year')['Year'].unique().tolist()
    return output
    
# Hold area and indicator the same (index level is held because unemployment appears at two levels)    
diff_output = df.groupby(['Area Name', 'Index level', 'Indicator/grouping name']).apply(get_diffs)

In [7]:
# Populate the original df with 1 year change values
def grab_diff(row):
    area_name = row['Area Name']
    index_level = row['Index level']
    indicator_grouping_name = row['Indicator/grouping name']
    year = row['Year']
    # diff_output is a multi-index table, hence needing three variables in first index position
    return diff_output.loc[area_name, index_level, indicator_grouping_name][year]

df['Change1year'] = df.apply(grab_diff, axis=1)
df['Change1year'] = df['Change1year'].round(1)

In [8]:
# Create a table that holds the change in value from 3 years ago (for 2018 only)
def get_3_year_diff(group):
    output = group.sort_values('Year')['Index value']
    return output.iloc[-1] - output.iloc[0]
    
three_year_diff_output = df.groupby(['Area Name', 'Index level', 'Indicator/grouping name']).apply(get_3_year_diff)

In [9]:
# Populate the original df with 3 year change values
def grab_3_year_diff(row):
    year = row['Year']
    # Only return a number if we are looking at data for 2018
    if year == 2018:
        area_name = row['Area Name']
        index_level = row['Index level']
        indicator_grouping_name = row['Indicator/grouping name']
        return three_year_diff_output.loc[area_name, index_level, indicator_grouping_name]
    else:
        return np.nan
df['Change3year'] = df.apply(grab_3_year_diff, axis=1)
df['Change3year'] = df['Change3year'].round(1)

## Create rankings for index value, one year change, and three year change

In [10]:
# Change NaNs to string "NaNs" so it is JSON compatible
df = df.fillna("NaN")

In [11]:
# Function creates sorted list of index values for grouped dataframe
def create_rankings(group):
    
    # Create a sorted list of all index values within group
    index_list = group["Index value"].tolist()
    index_list.sort(reverse = True)
    group["Rank"] = group.apply(input_ranking, index_list=index_list, compare="Index value", axis=1)
    
    # Create list for one year change
    index_list_1year = group["Change1year"].tolist()
    index_list_1year.sort(reverse = True)
    group["Change1year Rank"] = group.apply(input_ranking, index_list=index_list_1year, compare="Change1year", axis=1)

    # Create list for three year change
    index_list_3year = group["Change3year"].tolist()
    index_list_3year.sort(reverse = True)
    group["Change3year Rank"] = group.apply(input_ranking, index_list=index_list_3year, compare="Change3year", axis=1)

    return group

# Function is called by the create_rankings function and outputs rank per row
def input_ranking(row, index_list, compare):
    # Find the position of the ith value within the list and add one to give the first rank a value of 1
    ind_value = row[compare]
    # Don't rank NaNs and also omit top ranks for 0 change
    if (ind_value=="NaN")|(ind_value==0):
        rank = "NaN"
    else:
        rank = index_list.index(ind_value) + 1

    return rank

# Hold 'Geography type', 'Year', 'Indicator/grouping name', 'Index level' to compare areas
df = df.groupby(['Geography type', 'Year', 'Indicator/grouping name', 'Index level']).apply(create_rankings)

In [12]:
# Create object holding number of areas of each geog type
area_amounts = {"National": len(df[(df["Geography type"]=="National") & (df["Year"]==2018) & (df["Indicator/grouping name"]=="Overall")]), "Region": len(df[(df["Geography type"]=="Region") & (df["Year"]==2018) & (df["Indicator/grouping name"]=="Overall")]), "Upper Tier Local Authority": len(df[(df["Geography type"]=="Upper Tier Local Authority") & (df["Year"]==2018) & (df["Indicator/grouping name"]=="Overall")])}

### Of the three rank types, which is the **highest** for each row of data

In [13]:
# Create a column populated by the highest of the three ranks
def highest_rank(row):
    row = row.replace("NaN", 400)
    
    if (row['Rank'] <= row['Change1year Rank']) and (row['Rank'] <= row['Change3year Rank']):
        highest = row['Rank']
    elif (row['Change1year Rank'] <= row['Rank']) and (row['Change1year Rank'] <= row['Change3year Rank']):
        highest = row['Change1year Rank']
    elif (row['Change3year Rank'] <= row['Rank']) and (row['Change3year Rank'] <= row['Change1year Rank']):
        highest = row['Change3year Rank']
    return highest 

df['highestRank'] = df.apply(highest_rank, axis=1)

In [14]:
# Create a column populated by the type of the highest rank
def highest_rank_type(row):
    row = row.replace("NaN", 400)
    
    if (row['Rank'] <= row['Change1year Rank']) and (row['Rank'] <= row['Change3year Rank']):
        type_rank = 'Rank'
    elif (row['Change1year Rank'] <= row['Rank']) and (row['Change1year Rank'] <= row['Change3year Rank']):
        type_rank = 'Change1year Rank'
    elif (row['Change3year Rank'] <= row['Rank']) and (row['Change3year Rank'] <= row['Change1year Rank']):
        type_rank = 'Change3year Rank'
    return type_rank 

df['highestRankType'] = df.apply(highest_rank_type, axis=1)

### Of the three rank types, which has the **lowest** rank for each row

In [15]:
# Create a column populated by the lowest of the three ranks
def lowest_rank(row):
    row = row.replace("NaN", 0)
    
    if (row['Rank'] >= row['Change1year Rank']) and (row['Rank'] >= row['Change3year Rank']):
        lowest = row['Rank']
    elif (row['Change1year Rank'] >= row['Rank']) and (row['Change1year Rank'] >= row['Change3year Rank']):
        lowest = row['Change1year Rank']
    elif (row['Change3year Rank'] >= row['Rank']) and (row['Change3year Rank'] >= row['Change1year Rank']):
        lowest = row['Change3year Rank']
    return (1+area_amounts[row["Geography type"]]) - lowest 

df['lowestRank'] = df.apply(lowest_rank, axis=1)

In [16]:
# Create a column populated by the lowest of the three ranks
def lowest_rank_type(row):
    row = row.replace("NaN", 0)
    
    if (row['Rank'] >= row['Change1year Rank']) and (row['Rank'] >= row['Change3year Rank']):
        type_rank = 'Rank'
    elif (row['Change1year Rank'] >= row['Rank']) and (row['Change1year Rank'] >= row['Change3year Rank']):
        type_rank = 'Change1year Rank'
    elif (row['Change3year Rank'] >= row['Rank']) and (row['Change3year Rank'] >= row['Change1year Rank']):
        type_rank = 'Change3year Rank'
    return type_rank 

df['lowestRankType'] = df.apply(lowest_rank_type, axis=1)

In [17]:
# Create columns containing high/low rank data with unified title for ease of JS indexing
def high_low(row):
    if row["lowestRank"] < row["highestRank"]:
        highest = row["lowestRank"]
    elif row["lowestRank"] >= row["highestRank"]:
        highest = row["highestRank"]
    return highest
df['hlRank'] = df.apply(high_low, axis=1)

def high_low_type(row):
    if row["lowestRank"] < row["highestRank"]:
        highest = row["lowestRankType"]
    elif row["lowestRank"] >= row["highestRank"]:
        highest = row["highestRankType"]
    return highest
df['hlRankType'] = df.apply(high_low_type, axis=1)

## Create a nested dictionary which will be exported as a JSON file

In [36]:
def produce_json_for_area_code(group):
    output_dict = {}
    
    # Base level info about the selected area
    output_dict['area'] = group.name
    output_dict['name'] = group['Area Name'].iloc[0]
    output_dict['type'] = group['Geography type'].iloc[0]
    
    if group['Area Name'].iloc[0] == 'Manchester':
        dfTemp = group[group["Indicator/grouping name"]=="Mortality"]
        print("*******")
        print(dfTemp[dfTemp["Year"]==2018])
        print("*******")
        print(dfTemp[dfTemp["Year"]==2018].sort_values('highestRank'))
        print("*******")
        print(dfTemp[dfTemp["Year"]==2018].sort_values('highestRank')[['Measure', 'highestRank', 'highestRankType', 'Index level', 'Domain', 'Subdomain', 'Indicator', 'value', 'Change1year', 'Change3year']])
        print("*******")
        print(dfTemp[dfTemp["Year"]==2018].sort_values('highestRank')[['Measure', 'highestRank', 'highestRankType', 'Index level', 'Domain', 'Subdomain', 'Indicator', 'value', 'Change1year', 'Change3year']].to_dict(orient='records')[:20])
    
    # A base level object which contains data about the top and bottom ranked indicators for each area
    output_dict['priority2018'] = {}
    pri_group_high = group[group["Year"]==2018].sort_values('highestRank')
    pri_group_low = group[group["Year"]==2018].sort_values('lowestRank')
    output_dict['priority2018']["Highest"] = pri_group_high[['Measure', 'highestRank', 'highestRankType', 'Index level', 'Domain', 'Subdomain', 'Indicator', 'value', 'Change1year', 'Change3year']].to_dict(orient='records')[:20]
    output_dict['priority2018']["Lowest"] = pri_group_low[['Measure', 'lowestRank', 'lowestRankType', 'Index level', 'Domain', 'Subdomain', 'Indicator', 'value', 'Change1year', 'Change3year']].to_dict(orient='records')[:20]
    
    # The nested object containing all the data with hierarchal structure
    output_dict['data'] = {}
    group = group.set_index('Index level')
    
    # Create an object for each of the three domains, plus 'overall'
    for domain in group['Domain'].unique():
        output_dict['data'][domain] = {}
        
        # Data for the total values of each domain 
        domain_level_totals = group[group['Indicator/grouping name'] == domain].sort_values('Year').set_index('Year')
        output_dict['data'][domain]['total'] = domain_level_totals[['value', 'Rank', 'Change1year', 'Change3year', 'Change1year Rank', 'Change3year Rank']].to_dict(orient='index')
        
        # Besides 'total' object create 'subdomains'
        output_dict['data'][domain]['subdomains'] = {}
        
        # Find each subdomains associated with this domain
        rows_with_corresponding_domain = group[group['Domain'] == domain]
        for subdomain in rows_with_corresponding_domain['Subdomain'].unique():
            if subdomain != 'NaN':
                
                # Add data for the total value of this subdomain 
                subdomain_level_totals = group[(group['Indicator/grouping name'] == subdomain) & (group['Indicator']=="NaN")].sort_values('Year').set_index('Year') 
                output_dict['data'][domain]['subdomains'][subdomain] = {}
                output_dict['data'][domain]['subdomains'][subdomain]['total'] = subdomain_level_totals[['value', 'Rank', 'Change1year', 'Change3year', 'Change1year Rank', 'Change3year Rank']].to_dict(orient='index')
                
                # Find the rows for indicators within this subdomain
                rows_with_corresponding_subdomain = group[group['Subdomain'] == subdomain]
                subset = rows_with_corresponding_subdomain[rows_with_corresponding_subdomain['Indicator']!="NaN"][['Indicator', 'value', 'Rank', 'Change1year', 'Change3year', 'Change1year Rank', 'Change3year Rank', 'Year']].set_index(['Indicator', 'Year'])
                
                # Store indicator level data within the subdomain
                output_dict['data'][domain]['subdomains'][subdomain]['indicators'] = subset.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict(orient='index')).to_dict()
    
    return output_dict

data_dict = df.groupby('Area Code').apply(produce_json_for_area_code)

*******
      Area Code   Area Name              Geography type  Year Index level  \
7631  E08000003  Manchester  Upper Tier Local Authority  2018   Subdomain   

     Indicator/grouping name  Index value  Rank Change1year Change3year  \
7631               Mortality         78.3   148         3.0         4.0   

      highestRank   highestRankType  lowestRank lowestRankType  \
7631            8  Change1year Rank           2           Rank   

     Change1year Rank Change3year Rank  value    Measure          Domain  \
7631                8               14   78.3  Mortality  Healthy People   

      Subdomain Indicator  hlRank hlRankType  
7631  Mortality       NaN       2       Rank  
*******
      Area Code   Area Name              Geography type  Year Index level  \
7631  E08000003  Manchester  Upper Tier Local Authority  2018   Subdomain   

     Indicator/grouping name  Index value  Rank Change1year Change3year  \
7631               Mortality         78.3   148         3.0         

### Save each area as a seperate JSON file
Change the root file path

In [41]:
for i in range(len(data_dict)):
    with open('/Users/theojolliffe/Documents/healthindexlads/'+data_dict[i]['area']+'.json', 'w') as outfile:
        json.dump(data_dict[i], outfile)
        

In [69]:
!git add .

In [70]:
!git commit -m "Add files"

[main a3ab5b2] Add files
 20 files changed, 3146 insertions(+), 18 deletions(-)
 create mode 100644 .ipynb_checkpoints/Physiological risk factors-checkpoint.csv
 create mode 100644 Access to green space.csv
 create mode 100644 Access to housing.csv
 create mode 100644 Access to services.csv
 create mode 100644 Behavioural risk factors.csv
 create mode 100644 Children and young people's education.csv
 create mode 100644 Crime.csv
 create mode 100644 Difficulties in daily life.csv
 create mode 100644 Local enviroment.csv
 create mode 100644 Mental health.csv
 create mode 100644 Mortality.csv
 create mode 100644 Personal well-being.csv
 create mode 100644 Physical health conditions.csv
 create mode 100644 Physiological risk factors.csv
 create mode 100644 Protective measures.csv
 create mode 100644 Risk factors for children.csv
 create mode 100644 Unemployment.csv
 create mode 100644 Working conditions.csv


In [71]:
!git push -u origin main

Enumerating objects: 26, done.
Counting objects: 100% (26/26), done.
Delta compression using up to 16 threads
Compressing objects: 100% (22/22), done.
Writing objects: 100% (22/22), 24.21 KiB | 3.03 MiB/s, done.
Total 22 (delta 19), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (19/19), completed with 3 local objects.
To https://github.com/theojolliffe/healthindexlads.git
   1f4cd57..a3ab5b2  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [36]:
df = pd.read_csv('/Users/theojolliffe/Downloads/All data-Table.csv')
df = df.drop(['Unnamed: 7'], axis=1)
set(df['Geography type'])

{'National', 'Region', 'Upper Tier Local Authority'}

In [ ]:
parents = pd.read_csv('/Users/theojolliffe/Documents/healthindexlads/csv/includedHealthAreas.csv')
for (i, j) in enumerate(dfTemp['Area Name']):
    num = i+1
    if j not in set(parents['name']):
        print(j)
        parents.loc[-num] = [0, 0, j, 'NaN', 'lad', 'unk']

In [63]:
subdomains = df[(df['Index level']=='Subdomain')&(df['Year']==2018)&(df['Geography type']=='Upper Tier Local Authority')]
subdomains['parents'] = [parents[parents['name']==i].iloc[0]['parent'] for i in subdomains['Area Name']]
subdomains

<ipython-input-63-ef6d46aa7e58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdomains['parents'] = [parents[parents['name']==i].iloc[0]['parent'] for i in subdomains['Area Name']]


,Area Code,Area Name,Geography type,Year,Index level,Indicator/grouping name,Index value,parents
3227,E06000001,Hartlepool,Upper Tier Local Authority,2018,Subdomain,Physiological risk factors,84.6,E12000001
3231,E06000002,Middlesbrough,Upper Tier Local Authority,2018,Subdomain,Physiological risk factors,96.2,E12000001
3235,E06000003,Redcar and Cleveland,Upper Tier Local Authority,2018,Subdomain,Physiological risk factors,79.2,E12000001
3239,E06000004,Stockton-on-Tees,Upper Tier Local Authority,2018,Subdomain,Physiological risk factors,91.5,E12000001
3243,E06000005,Darlington,Upper Tier Local Authority,2018,Subdomain,Physiological risk factors,87.2,E12000001
...,...,...,...,...,...,...,...,...
13339,E10000029,Suffolk,Upper Tier Local Authority,2018,Subdomain,Crime,115.3,unk
13343,E10000030,Surrey,Upper Tier Local Authority,2018,Subdomain,Crime,99.6,unk
13347,E10000031,Warwickshire,Upper Tier Local Authority,2018,Subdomain,Crime,114.9,unk
13351,E10000032,West Sussex,Upper Tier Local Authority,2018,Subdomain,Crime,99.8,unk


In [68]:
for i in set(subdomains['Indicator/grouping name']):
    dfTemp = subdomains[subdomains['Indicator/grouping name']==i]
    dfTemp.to_csv('/Users/theojolliffe/Documents/healthindexlads/'+i+'.csv', index=False)